In [3]:
import mlflow
mlflow.set_tracking_uri("file:/Users/aminatafall/mlruns")  # Chemin complet
mlflow.set_experiment("LogisticRegression")



<Experiment: artifact_location='file:///Users/aminatafall/mlruns/362138514271581840', creation_time=1742571592919, experiment_id='362138514271581840', last_update_time=1742571592919, lifecycle_stage='active', name='LogisticRegression', tags={}>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

import mlflow
import mlflow.sklearn

output_dir = "/tmp/mlflow_outputs"
os.makedirs(output_dir, exist_ok=True)

mlflow.set_tracking_uri("http://localhost:5001")

mlflow.set_experiment("LogisticRegression-Loan")

df = pd.read_csv("Loan_Data.csv")

X = df.drop(['customer_id', 'default'], axis=1)  
y = df['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = [
    {"C": 1.0, "solver": "liblinear"},
    {"C": 0.1, "solver": "liblinear"},
    {"C": 10.0, "solver": "lbfgs", "max_iter": 1000},
]

for params in param_grid:
    with mlflow.start_run():
        print(f"\n➡️ Training LogisticRegression avec params: {params}")
        
        # Modèle
        model = LogisticRegression(**params)
        model.fit(X_train, y_train)
        
        # Prédiction
        y_pred = model.predict(X_test)
        
        # Métriques
        acc = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        
        # Log hyperparamètres & métriques
        mlflow.log_params(params)
        mlflow.log_metric("Accuracy", acc)
        mlflow.log_metric("Precision", precision)
        mlflow.log_metric("Recall", recall)
        
        
        mlflow.sklearn.log_model(model, "model")
        
       
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel("Prédictions")
        plt.ylabel("Vraies valeurs")
        plt.title("Matrice de confusion")
        plt.tight_layout()
        
      
        image_path = os.path.join(output_dir, "confusion_matrix.png")
        plt.savefig(image_path)
        plt.close()
        
        # Log dans MLflow
        mlflow.log_artifact(image_path)
        os.remove(image_path)  # Nettoyage après log

print(" Tous les runs sont loggés dans MLflow. Vérifie sur : http://localhost:5001")


2025/03/22 13:15:54 INFO mlflow.tracking.fluent: Experiment with name 'LogisticRegression-Loan' does not exist. Creating a new experiment.



➡️ Training LogisticRegression avec params: {'C': 1.0, 'solver': 'liblinear'}


2025/03/22 13:15:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bright-crane-123 at: http://localhost:5001/#/experiments/790845414201742872/runs/d4ca106c86084dea8ff43c6a06c1dff9
🧪 View experiment at: http://localhost:5001/#/experiments/790845414201742872

➡️ Training LogisticRegression avec params: {'C': 0.1, 'solver': 'liblinear'}


2025/03/22 13:16:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run mercurial-tern-736 at: http://localhost:5001/#/experiments/790845414201742872/runs/3b74402b6f1b4029af35148220049cff
🧪 View experiment at: http://localhost:5001/#/experiments/790845414201742872

➡️ Training LogisticRegression avec params: {'C': 10.0, 'solver': 'lbfgs', 'max_iter': 1000}


2025/03/22 13:16:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run stately-ape-887 at: http://localhost:5001/#/experiments/790845414201742872/runs/6d08b7c1c0b7420e97c33a5ec1b483f4
🧪 View experiment at: http://localhost:5001/#/experiments/790845414201742872
 Tous les runs sont loggés dans MLflow. Vérifie sur : http://localhost:5001
